# CatBoost with Categorical Features + Interaction Terms

**CRITICAL INSIGHT**: The CV-LB relationship has intercept (0.0533) > target (0.0347).
- Even with CV = 0, LB would be 0.0533
- We CANNOT reach the target by improving CV alone
- We need to CHANGE THE CV-LB RELATIONSHIP

**Hypothesis**: CatBoost handles categorical features natively and might have different extrapolation behavior.

**Implementation**:
- Use solvent name as categorical feature (not one-hot encoded)
- Add interaction features: Reaction_Energy = Temperature * Residence Time
- CatBoost uses ordered boosting which might generalize better
- Train 3 separate models for each target

In [ ]:
# Standard imports
import numpy as np
import pandas as pd
import torch
from catboost import CatBoostRegressor
from sklearn.preprocessing import StandardScaler
import tqdm
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
torch.manual_seed(42)

print('Imports complete')

In [ ]:
# Data loading functions
DATA_PATH = '/home/data'

INPUT_LABELS_NUMERIC = ["Residence Time", "Temperature"]
INPUT_LABELS_SINGLE_SOLVENT = ["Residence Time", "Temperature", "SOLVENT NAME"]
INPUT_LABELS_FULL_SOLVENT = ["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]

def load_data(name="full"):
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[INPUT_LABELS_FULL_SOLVENT]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[INPUT_LABELS_SINGLE_SOLVENT]
    Y = df[["Product 2", "Product 3", "SM"]]
    return X, Y

def generate_leave_one_out_splits(X, Y):
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = ~((X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X["SOLVENT B NAME"] == row["SOLVENT B NAME"]))
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

print('Data loading functions defined')

In [ ]:
# Load feature lookups for Spange descriptors
SPANGE_DF = pd.read_csv(f'{DATA_PATH}/spange_descriptors_lookup.csv', index_col=0)
print(f'Spange: {SPANGE_DF.shape}')

In [ ]:
# CatBoost Model with Categorical Features and Interaction Terms
class CatBoostModel:
    """CatBoost model with native categorical feature handling.
    
    Key differences from previous approaches:
    1. Solvent name treated as categorical (not one-hot encoded)
    2. CatBoost uses ordered boosting which might generalize better
    3. Interaction features capture domain knowledge
    """
    def __init__(self, data='single'):
        self.data_type = data
        self.models = []  # One per target
        self.spange_df = SPANGE_DF
        self.y_scaler = StandardScaler()
        
    def _create_features(self, X):
        """Create features with interaction terms and Spange descriptors."""
        df = X.copy().reset_index(drop=True)
        
        # Kinetic features
        df['inv_temp'] = 1000.0 / (df['Temperature'] + 273.15)
        df['log_time'] = np.log(df['Residence Time'] + 1e-6)
        df['interaction'] = df['inv_temp'] * df['log_time']
        
        # Interaction features (domain knowledge)
        df['Reaction_Energy'] = df['Temperature'] * df['Residence Time']
        
        if self.data_type == 'full':
            # For mixed solvents
            df['B_Conc_Temp'] = df['SolventB%'] * df['Temperature']
            
            # Get Spange descriptors for both solvents
            A_spange = self.spange_df.loc[X["SOLVENT A NAME"]].values
            B_spange = self.spange_df.loc[X["SOLVENT B NAME"]].values
            pct = X["SolventB%"].values.reshape(-1, 1)
            
            # Weighted average of Spange descriptors
            mixed_spange = A_spange * (1 - pct) + B_spange * pct
            
            # Add Spange features
            for i, col in enumerate(self.spange_df.columns):
                df[f'spange_{col}'] = mixed_spange[:, i]
            
            # Create categorical feature for solvent pair
            df['solvent_cat'] = X["SOLVENT A NAME"].values + "_" + X["SOLVENT B NAME"].values
            
            # Select features for CatBoost
            feature_cols = ['Residence Time', 'Temperature', 'SolventB%', 
                           'inv_temp', 'log_time', 'interaction',
                           'Reaction_Energy', 'B_Conc_Temp', 'solvent_cat'] + \
                          [f'spange_{col}' for col in self.spange_df.columns]
        else:
            # For single solvents
            # Get Spange descriptors
            spange_vals = self.spange_df.loc[X["SOLVENT NAME"]].values
            
            # Add Spange features
            for i, col in enumerate(self.spange_df.columns):
                df[f'spange_{col}'] = spange_vals[:, i]
            
            # Create categorical feature for solvent
            df['solvent_cat'] = X["SOLVENT NAME"].values
            
            # Select features for CatBoost
            feature_cols = ['Residence Time', 'Temperature',
                           'inv_temp', 'log_time', 'interaction',
                           'Reaction_Energy', 'solvent_cat'] + \
                          [f'spange_{col}' for col in self.spange_df.columns]
        
        return df[feature_cols]
    
    def train_model(self, X, Y):
        """Train CatBoost models (one per target)."""
        X_feat = self._create_features(X)
        Y_np = Y.values
        
        # Scale targets
        Y_scaled = self.y_scaler.fit_transform(Y_np)
        
        # Find categorical column index
        cat_features = ['solvent_cat']
        
        # Train 3 separate CatBoost models (one per target)
        self.models = []
        for i in range(3):
            model = CatBoostRegressor(
                iterations=500,
                learning_rate=0.05,
                depth=6,
                l2_leaf_reg=3,
                cat_features=cat_features,
                random_seed=42,
                verbose=False
            )
            model.fit(X_feat, Y_scaled[:, i])
            self.models.append(model)
    
    def predict(self, X):
        """Predict using trained CatBoost models."""
        X_feat = self._create_features(X)
        
        # Get predictions from each model
        preds_scaled = np.column_stack([model.predict(X_feat) for model in self.models])
        
        # Inverse transform
        preds = self.y_scaler.inverse_transform(preds_scaled)
        
        # Clip to valid range
        preds = np.clip(preds, 0, 1)
        
        return torch.tensor(preds)
    
    def predict_with_tta(self, X):
        """Predict with test-time augmentation for mixed solvents."""
        if self.data_type == 'single':
            return self.predict(X)
        
        # For mixed solvents, average predictions from both orderings
        pred1 = self.predict(X)
        
        # Create flipped version
        X_flip = X.copy()
        X_flip["SOLVENT A NAME"] = X["SOLVENT B NAME"]
        X_flip["SOLVENT B NAME"] = X["SOLVENT A NAME"]
        X_flip["SolventB%"] = 1 - X["SolventB%"]
        
        pred2 = self.predict(X_flip)
        
        # Average
        return (pred1 + pred2) / 2

print('CatBoostModel defined with categorical features and interaction terms')

In [ ]:
# Test the model
X_single, Y_single = load_data("single_solvent")
print(f"Single solvent data: {X_single.shape[0]} samples, {len(X_single['SOLVENT NAME'].unique())} unique solvents")

X_full, Y_full = load_data("full")
ramps = X_full[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
print(f"Full data: {X_full.shape[0]} samples, {len(ramps)} unique ramps")

# Quick test
model = CatBoostModel(data='single')
model.train_model(X_single.head(100), Y_single.head(100))
pred = model.predict(X_single.head(10))
print(f"Test prediction shape: {pred.shape}")

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

import tqdm

X, Y = load_data("single_solvent")

split_generator = generate_leave_one_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = CatBoostModel(data='single')  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 0,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_single_solvent = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

X, Y = load_data("full")

split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = CatBoostModel(data='full')  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict_with_tta(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 1,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_full_data = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

submission = pd.concat([submission_single_solvent, submission_full_data])
submission = submission.reset_index()
submission.index.name = "id"
submission.to_csv("/home/submission/submission.csv", index=True)

########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [ ]:
# Calculate CV score (for verification only - NOT part of submission)
X_single, Y_single = load_data("single_solvent")
X_full, Y_full = load_data("full")

# Get actuals in same order as predictions
actuals_single = []
for solvent in sorted(X_single["SOLVENT NAME"].unique()):
    mask = X_single["SOLVENT NAME"] == solvent
    actuals_single.append(Y_single[mask].values)
actuals_single = np.vstack(actuals_single)

actuals_full = []
ramps = X_full[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
for _, row in ramps.iterrows():
    mask = (X_full["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X_full["SOLVENT B NAME"] == row["SOLVENT B NAME"])
    actuals_full.append(Y_full[mask].values)
actuals_full = np.vstack(actuals_full)

# Get predictions
preds_single = submission_single_solvent[['target_1', 'target_2', 'target_3']].values
preds_full = submission_full_data[['target_1', 'target_2', 'target_3']].values

# Calculate MSE
mse_single = np.mean((actuals_single - preds_single) ** 2)
mse_full = np.mean((actuals_full - preds_full) ** 2)
n_single = len(actuals_single)
n_full = len(actuals_full)
overall_mse = (mse_single * n_single + mse_full * n_full) / (n_single + n_full)

print(f'\n=== CATBOOST CV SCORE ===')
print(f'Single Solvent MSE: {mse_single:.6f} (n={n_single})')
print(f'Full Data MSE: {mse_full:.6f} (n={n_full})')
print(f'Overall MSE: {overall_mse:.6f}')

print(f'\n=== COMPARISON ===')
print(f'exp_032 (best CV, GP 0.15 + MLP 0.55 + LGBM 0.3): CV 0.008194')
print(f'exp_047 (CATBOOST): CV {overall_mse:.6f}')

if overall_mse < 0.008194:
    improvement = (0.008194 - overall_mse) / 0.008194 * 100
    print(f'\n✓ IMPROVEMENT: {improvement:.2f}% better than exp_032!')
else:
    degradation = (overall_mse - 0.008194) / 0.008194 * 100
    print(f'\n✗ WORSE: {degradation:.2f}% worse than exp_032')
    
# Estimate LB using old relationship
estimated_lb = 4.23 * overall_mse + 0.0533
print(f'\nEstimated LB (using old relationship): {estimated_lb:.4f}')
print(f'Best LB so far: 0.0877')
print(f'Target: 0.0347')
print(f'\nKey question: Does CatBoost have a DIFFERENT CV-LB relationship?')
print(f'If it has lower intercept, we might be able to reach the target.')